# Week 2

## Binary classification

We should organize the data into matrices. Putting training examples to columns (instead of rows) helps in deep learning.

<img src="notes_images/bicla.png" width="700">

## Logistic regression

We are not using the notation in red, but that might be used in other courses.

<img src="notes_images/logre.png" width="700">

## Logistic regression cost function

Loss function is applied to a single training example. Cost function is an average over the entire set, it is the cost of parameters that we try to minimize.

We use this cost function since it's convex. This means it does not have a bunch a local optima (like non-convex functions) and it's easy to minimize.

<img src="notes_images/log_cost.png" width="700">

## Gradient descent

We are minimizing the cost function with the gradient descent algorithm.

Refresher: *Partial derivative* (symbol) is used when J is a function of multiple variables and we are deriving it with respect to one of those variables (holding the others constant). For *total derivative* we would derive it with respect to all the variables.

In the forward propagation step of the neural network we calculate the outputs. In backward propagation we calculate the gradients.

<img src="notes_images/grade.png" width="700">

## Logistic regression gradient descent

In backpropagation we step backwards through the network calculating numeric (?) derivatives along the way. We want to reach the parameters w1, w2 and b. Once we know how they affect the cost function, we can change their values in a way that decreases the CF.

<img src="notes_images/logre_back.png" width="700">

## Gradient descent on *m* training examples

To implement a single step of gradient descent, we can do as in the following figure. However, we actually want to use vectorization instead of for-loops.

<img src="notes_images/grade_multi.png" width="700">

## Python and vectorization

In [2]:
import numpy as np

a = np.random.rand(100)
b = np.random.rand(100)

In [15]:
%%timeit
np.sum(a * b)

5.2 µs ± 414 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
%%timeit
np.dot(a, b)

1.03 µs ± 36.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


<img src="notes_images/logre_vect.png" width="700">

#### Broadcasting is nice and fun:

In [39]:
a = np.random.rand(10).reshape(5, 2)
a

array([[0.95364508, 0.84156954],
       [0.81193213, 0.68918493],
       [0.84205848, 0.86530224],
       [0.91783936, 0.29015515],
       [0.49188914, 0.09640765]])

In [40]:
a / np.array([1, 2])

array([[0.95364508, 0.42078477],
       [0.81193213, 0.34459246],
       [0.84205848, 0.43265112],
       [0.91783936, 0.14507757],
       [0.49188914, 0.04820383]])

#### Do not use rank one arrays, they mess things up:

In [58]:
# So not this (5, ) shaped stuff
a = np.random.rand(5)
print(a)
print(a.shape)
print(a.T)

print(np.dot(a, a.T))

[0.74044184 0.33195623 0.20609904 0.0765147  0.00702446]
(5,)
[0.74044184 0.33195623 0.20609904 0.0765147  0.00702446]
0.7068297026170908


In [59]:
# Instead, reshape or define it in rand
# b = np.random.rand(5, 1)
# print(b)
# print(b.shape)

c = np.random.rand(5).reshape(-1, 1)
print(c)
print(c.shape)
print(np.dot(c, c.T))

[[0.64263608]
 [0.8886038 ]
 [0.89813282]
 [0.8692878 ]
 [0.93195082]]
(5, 1)
[[0.41298114 0.57104887 0.57717256 0.55863571 0.59890522]
 [0.57104887 0.78961672 0.79808424 0.77245245 0.82813504]
 [0.57717256 0.79808424 0.80664257 0.78073591 0.83701562]
 [0.55863571 0.77245245 0.78073591 0.75566128 0.81013348]
 [0.59890522 0.82813504 0.83701562 0.81013348 0.86853233]]


In [62]:
# Can use assert to make sure shape is correct
assert(c.shape == (5, 1))

In [63]:
assert(a.shape == (5, 1))

AssertionError: 

In [6]:
a = np.random.randn(12288, 150) # a.shape = (12288, 150)
b = np.random.randn(150, 45) # b.shape = (150, 45)
c = np.dot(a,b)
c.shape

(12288, 45)

## Week 3

<img src="notes_images/whatis_nn.png" width="700">

Hidden layer is *hidden*, because its values are not in the training set. In contrast, the input layer and the output layer both have "values" in the training set.

The nn below is a two-layer nn, because the input layer is not counted as an "official" layer.

<img src="notes_images/nn_repres.png" width="700">

<img src="notes_images/nn_repres2.png" width="700">

<img src="notes_images/nn_repres3.png" width="700">

<img src="notes_images/nn_repres4.png" width="700">

The equations above are all that is needed for creating a 2 layer neural network structure (forward prop). Note, in the above example, we just have one training example (with three features).

Vectorization is easy as long as we stack all the *m* training examples into a single matrix *X* (as per usual).

<img src="notes_images/nn_vector.png" width="700">

Generally, with our notation the matrix rows correspond to different neurons (or features for input layer) and the columns correspond to different training examples.

Here is a simple showcase for why the vectorization works:

<img src="notes_images/vect_just.png" width="700">

We have been using the sigmoid function so far for the activation. However, this is not the only or the best choice. For example *tanh* almost always works better than the sigmoid function, since it "centers" the data (mean) at zero, instead of at 0.5, which makes learning easier for the next layer. This will be discussed in more detail in the next course.

Probably the only reason to use sigmoid over tanh, is for binary classifications output layer, where y E [0, 1]

However, both sigmoid and tanh have a problem: if *z* is very large or small, the derivative (gradient) of the function is close to zero, which slows down gradient descent.

Often, the superior alternative is the rectified linear unit, *ReLU*, function. Here, the derivative is one, when z is positive and zero, when z is negative. This works very well in practice most of the time. There is also another version, *leaky ReLU*, where the derivative is slightly positive, when z is negative. The degree of positivity can be decided or even optimized as a parameter, below it is set as 0.01. However, the standard ReLU is pretty much the default choice nowadays.

Rules of thumb:
1) If the output y E [0, 1], use sigmoid for output layer  
2) For all other units, use ReLU

So, **sigmoid < tanh < ReLU (= leaky ReLU)**. **But** can always test them all, since some of them can work really well for certain applications.

<img src="notes_images/act_funcs.png" width="700">

Non-linear activation functions are used, because without them a neural network is basically an overly complex linear model. Non-linear activations allow the network to create much more complex calculations and connections. In other words, just setting a = z = Wx + b for every layer and node makes very little sense and just makes the whole network a big dumb linear model.

Pretty much the only situation, where a linear activation a = z might be useful, is in the output layer of a regression problem (continuous output), like predicting housing prices. Even then, the earlier layers would have e.g. ReLU or tanh activations and ReLU might be better for the output layer too, since the prices > 0.

<img src="notes_images/nonl_act.png" width="700">

Derivatives for the different activations functions:

**sigmoid**  
$g'(z) = \frac{d}{dz} g(z) = g(z)(1 - g(z)) = a(1 - a)$  
**tanh**  
$g'(z) = 1 - tanh(z)^2 = 1 - a^2$  
**ReLU**  
$g'(z) = 0$, if $z < 0$    
$g'(z) = 1$, if $z > 0$  
$g'(z) =$ undefined, if $z = 0$  
However, in practice we can set the following, since a computer will never hit exactly 0 due to floats  
$g'(z) = 1$, if $z >= 0$  
**leaky ReLU** (see above)  
$g'(z) = 0.01$, if $z < 0$    
$g'(z) = 1$, if $z > 0$ (practically $z >= 0$)  
$g'(z) =$ undefined, if $z = 0$  



Summary of the vectorized equations for the gradient descent of the current network:

<img src="notes_images/grade_summ.png" width="700">

For neural networks, we have to randomly initialize the parameters. If we just initialize them to zero, like previously with logistic regression, the network won't work. Specifically, the *W* terms cannot be initialized to zero (*b* is ok), because this makes the units symmetric and they will calculate exactly the same functions regardless of how many iterations we run gradient descent. So, the optimization gets "stuck" and never evolves. We need different hidden units calculating different things.

Typically, we initialize the weights to very small floats. They are small, since making them big can slow down gradient descent (especially if we have sigmoid functions, since the z-values will hit the tails). The exact constant value to use (here 0.01) depends on the depth of the network. For small applications, like the one we currently have, 0.01 is fine.

<img src="notes_images/nn_init.png" width="700">

## Week 4

A deep neural network has many hidden layers. But how many layers does it need to have to be considered "deep"? There is no definite answer. The opposite of a deep neural network is a "shallow" network with no or few hidden layers, such as logistic regression.

When first creating a nn to solve a problem, the amount of hidden layers should be considered a hyperparameter. Sometimes it's best to start with just log. reg. or just a couple of hidden layers and then scale up, if needed.

<img src="notes_images/deep_nn.png" width="700">

The forward propagation looks largely similar to what we had before. The only real difference is that in our application, we will have to define an explicit for-loop for going through the layers.

<img src="notes_images/dnn_forw.png" width="700">

A good way to hunt down bugs in nns is to take a piece of paper, go through the dimensions of all the matrices and see that they make sense.

A key thing to remember is that $W^{[l]}$ and $dW^{[l]}$ should have the same dimensions $[n^{l}, n^{l-1}]$. The same goes for $b^{[l]}$ and $db^{[l]}$ having $[n^{l}, 1]$.

Furthermore, with many training examples $A^{[l]}$ and $dA^{[l]}$ have dimensions $[n^{l}, m]$. These are the same for *Z* and *dZ*.

#### Why are deep neural networks working so well?

1) Typically, the first layers of a face-recognizing nn find simple features, like edges. Later layers might put these edges together to recognize parts of the face, like an eye. The last layers might recognize the entire face.

2) A deep nn generally requires way less hidden units than a shallow nn for solving the same problem.

While calculating forward prop, we cache the *z* (and *W*, *b*) values since they are used in backprop.

One iteration of training a nn (with gradient descent) includes forward prop to find y_hat and backword prop to find all the gradients, which we use to update the parameters. Then we forward prop again to get a new y_hat.

<img src="notes_images/dnn_f_b.png" width="700">

In the lecture slides, the star * is an elementwise multiplication. Also, in the slide below, the underlined a should be transposed.

<img src="notes_images/dnn_f_b_eq.png" width="700">

So this time backprop has 4 equations to implement. Forward prop works very similarly to before, no big surprises there.

The code for neural nets is actually relatively short, since **most of their power comes from having large datasets**.

Hyperparameters ultimately control the values of the parameters. Getting the hyperparameter values "right" often requires trial and error. So basically, you keep trying different values until you get a good enough result. There are systematic ways to do this, which are discussed later.

<img src="notes_images/dnn_hypar.png" width="700">

Neural nets have very little to do with actual human brains, even though the media loves simplifying them like that. The human brain is way more complicated, nobody understands how neurons in the human brain work.